In [180]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import re
import numpy as np
import pandas as pd

In [ ]:
%reset

In [198]:
urlList = ["https://www.zillow.com/lender-profile/PrimeLending","https://www.zillow.com/lender-profile/CaliberHomeLoans",
           "https://www.zillow.com/lender-profile/SupremeLendingCorporate","https://www.zillow.com/lender-profile/SupremeLendingCorporate",
          "https://www.zillow.com/lender-profile/Stearns%20Lending","https://www.zillow.com/lender-profile/Cendera%20Funding/",
          "https://www.zillow.com/lender-profile/Pacific%20Union/","https://www.zillow.com/lender-profile/Great%20Western/"
           ,"https://www.zillow.com/lender-profile/www.vipmtg.ccom/","https://www.zillow.com/lender-profile/zuser20151204075538567/","https://www.zillow.com/lender-profile/PrimeWest/"]
driver = webdriver.Chrome('C:/Users/Ali Prasla/chromedriver.exe')
allReviewList = []
for url in urlList:
    revList = []
    driver.get(url)
    #extract header data
    lenderName = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//h1[@class= "lender-name"]')))
    #showMoreReviewElem = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,'div[@class="zsg-content-item" and @style="text-align: center;"]')))
    lenderName = lenderName.text
    print lenderName
    time.sleep(1)
    divList = driver.find_elements_by_class_name("dont-break-out")
    header = divList[0].text
    #find show more reviews button:
    currentIdx = 0
    while True:
        aList = driver.find_elements_by_class_name('zsg-content-item')
        currentA = aList[currentIdx]
        #does element exist checker
        try:
            time.sleep(1)
            while currentA.text != "Show more reviews":
                currentA = aList[currentIdx]
                currentIdx += 1
            currentIdx -= 1
            currentA.click()
        except:
            break
        if currentIdx >= 1000:
            break
    #all scrolling completed by this point
    print "Begin processing reviews"
    allRevElement = driver.find_element_by_xpath("//ul[@class = 'zsg-content-component dont-break-out']")    
    print "Element found"
    revElementList = allRevElement.find_elements_by_xpath('//div[@style = "border-top: 1px solid rgb(204, 204, 204); padding-top: 20px;"]')
    print "revElementList created. Beginning for review iteration"
    print "Length of Element list", len(revElementList)
    while len(revElementList) > 0:
        #gets element from list
        reviewElement = revElementList.pop(0)
        rating  = reviewElement.find_elements_by_class_name('zsg-lg-1-2')[0].find_element_by_tag_name('span').get_attribute('class')
        rating = int(re.search("[0-9][0-9][0-9]",rating).group(0))/100.0
        l =  reviewElement.find_elements_by_class_name('zsg-fineprint')
        revDate = l[0].text
        location = l[1].text 
        try:
            location = re.search("[A-Z](.*),(.*)",location).group(0)
        except:
            location = np.NaN
        #use regex to extract location
        title = reviewElement.find_element_by_tag_name("h3").text
        textElement = reviewElement.find_element_by_class_name("dont-break-out")
        #clicks read more on reviews
        try:
            checkText = str(textElement.text)
        except:
            continue
        if "Read more" in checkText:
            #complicated, but executes javascript that deals with an 'Element not clickable' exception
            try:
                elem = textElement.find_element_by_tag_name('a')
                elem.click()
            except:
                continue
        reviewText = textElement.text
        revList.append([lenderName,rating,revDate,location,title,reviewText])
    allReviewList.extend(revList)
driver.close()

PrimeLending, A PlainsCapital Company
Begin processing reviews
Element found
revElementList created. Beginning for review iteration
Length of Element list 710
Caliber Home Loans
Begin processing reviews
Element found
revElementList created. Beginning for review iteration
Length of Element list 670
Supreme Lending
Begin processing reviews
Element found
revElementList created. Beginning for review iteration
Length of Element list 690
Supreme Lending
Begin processing reviews
Element found
revElementList created. Beginning for review iteration
Length of Element list 690
Stearns Lending, LLC
Begin processing reviews
Element found
revElementList created. Beginning for review iteration
Length of Element list 808
Cendera Funding, Inc.
Begin processing reviews
Element found
revElementList created. Beginning for review iteration
Length of Element list 841
Pacific Union Financial, LLC
Begin processing reviews
Element found
revElementList created. Beginning for review iteration
Length of Element l

In [199]:
print len(allReviewList[0])
df = pd.DataFrame(allReviewList)
df.columns = ["LenderName","Rating","ReviewDate","Location","Title","ReviewText"]
df.to_csv("ZillowLenderReviews.csv",encoding = 'utf-8')

6


In [203]:
df["LenderName"].value_counts()

Supreme Lending                           1358
Cendera Funding, Inc.                      835
Stearns Lending, LLC                       797
PrimeLending, A PlainsCapital Company      706
Caliber Home Loans                         659
Pacific Union Financial, LLC               592
VIP Mortgage Inc.                          426
Great Western Financial Services, Inc.     360
Certainty Home Loans, LLC                  334
PrimeWest Mortgage Corporation             322
Name: LenderName, dtype: int64